# IPAI SQL 

In [1]:
#imports
import pandas as pd
from pathlib import Path
import psycopg2

## Connect to DB

In [2]:
# being connected to wifi using fcul vpn
connection = psycopg2.connect(
database="ipai06",
user="ipai06",
password="ipai06",
host="appserver-01.alunos.di.fc.ul.pt",
port= '5432'
)

cursor = connection.cursor()

In [3]:
folder_path = 'dimensional_modeling'
folder = Path(folder_path)

filesList = []
# Iterate over the files in the folder
for file_path in folder.iterdir():
    if file_path.is_file():
        # Process the file
        filesList.append(file_path)

In [4]:
def createDIMTableSQL(df,fileName):
    tableName = fileName.split('.')[0]
    colNames = df.keys()
    createTabCmd = 'CREATE TABLE IF NOT EXISTS ' + tableName + ' ('
    
    for colName in colNames:
        if 'PK' in colName:
            substr = colName + ' INT PRIMARY KEY, '
        else:
            dtype = str(df[colName].dtypes)
            if 'int' in dtype:
                substr = colName + ' INT, '
            elif 'float' in dtype:
                substr = colName + ' REAL, '
            else:
                substr = colName + ' VARCHAR(255), '
        
        createTabCmd+= substr

    createTabCmd = createTabCmd[:-2]
    createTabCmd += ");"
    #return string

    return createTabCmd

In [5]:
def createFactTableSQL(df, fileName):
    tableName = fileName.split('.')[0]
    colNames = df.keys()
    createTabCmd = 'CREATE TABLE IF NOT EXISTS ' + tableName + ' ('

    for colName in colNames:
        if 'PK' in colName:
            substr = colName + ' INT PRIMARY KEY, '
        elif 'FK' in colName:
            substr = colName + ' INT REFERENCES '
            if 'nightPrice' in colName:
                substr += 'dimPriceAndBooking(nightPricePK), '
            elif 'street' in colName:
                substr += 'dimLocation(streetPK), '
            elif 'customer' in colName:
                substr += 'dim'+colName[:-2]+'(reviewerPK), '
            else:
                tableName =colName[:-2]
                pkColName = tableName+'PK'
                substr += 'dim'+tableName+'('+pkColName+'), '
        else:
            substr = colName + ' REAL NOT NULL, '

        createTabCmd+= substr

    createTabCmd = createTabCmd[:-2]
    createTabCmd += ");"

    return createTabCmd


In [6]:
def insertValuesIntoTableSQL(df, fileName):
    tableName = fileName.split('.')[0]
    insertValuesCmd = "INSERT INTO " + tableName + " values "
    listDF = list(df.itertuples(index=False, name=None))
    for item in listDF:
        itemStr = str(item)
        insertValuesCmd += itemStr + ', '
    insertValuesCmd = insertValuesCmd[:-2]
    insertValuesCmd += ";"
    return insertValuesCmd

In [7]:
def dimToSQL(file):
    df = pd.read_csv(file)
    table = createDIMTableSQL(df, file.name)
    cursor.execute(table)
    connection.commit()
    insertTable = insertValuesIntoTableSQL(df, file.name)
    cursor.execute(insertTable)
    connection.commit()


In [8]:
def factToSQL(file):
    df = pd.read_csv(file)
    table = createFactTableSQL(df, file.name)
    cursor.execute(table)
    connection.commit()
    insertTable = insertValuesIntoTableSQL(df, file.name)
    cursor.execute(insertTable)
    connection.commit()

In [20]:
connection.rollback()
for filePath in filesList:
    print(filePath)
    fileName = filePath.name
    if 'dim' in fileName:
        dimToSQL(filePath)
    elif 'Fact' in fileName:
        factToSQL(filePath)

dimensional_modeling\dimAmenities.csv
dimensional_modeling\dimCustomer.csv
dimensional_modeling\dimDate.csv
dimensional_modeling\dimFacilities.csv
dimensional_modeling\dimHost.csv
dimensional_modeling\dimLocation.csv
dimensional_modeling\dimPriceAndBooking.csv
dimensional_modeling\dimProperty.csv
dimensional_modeling\dimWeather.csv
dimensional_modeling\staysFact.csv


In [21]:

alterSQL = 'ALTER TABLE dimfacilities ADD CONSTRAINT fk_facilities_amenities FOREIGN KEY (amenitiesfk) REFERENCES dimamenities (amenitiespk);'
cursor.execute(alterSQL)
connection.commit()

In [4]:
# b+ index
bPlus_idx = 'CREATE INDEX month_index ON dimdate USING btree (month);'
cursor.execute(bPlus_idx)
connection.commit()

DuplicateTable: relation "month_index" already exists


In [20]:
# hash index
connection.rollback()
column_names_query = "SELECT column_name FROM information_schema.columns WHERE table_name = 'dimamenities' AND table_schema = 'public';"
cursor.execute(column_names_query)
result = list(cursor.fetchall())
col_names_list = [item for tuple in result for item in tuple]

for col in col_names_list:
    idx = col + '_index'
    col_idx = f"CREATE INDEX {idx} ON dimamenities USING hash ({col});"
    cursor.execute(col_idx)
    connection.commit()